## Chapter 03. BERT

#### BERT of Transformers

In [1]:
# Tensor flow용 Masked LM ; 문장 중간에 가려진 부분을 추측하는 모델
# Pytorch 용은 앞에 TF를 빼면 됨
from transformers import TFBertForMaskedLM, AutoTokenizer

# 모델 호출
model = TFBertForMaskedLM.from_pretrained('bert-base-uncased')
# uncased; 대소문자 구분을 하지 않음

# 토크나이저 호출
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

c:\Users\lucky\anaconda3\envs\TEST\lib\site-packages\huggingface_hub\file_download.py:147: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\lucky\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFBertForMaskedLM.

All the weights of TFBertForMaskedLM were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForMaskedLM for predictions without further training.


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [2]:
# BERT 모형에서 사용하는 mask_token (빈 칸)
tokenizer.mask_token

'[MASK]'

In [3]:
inputs = tokenizer('Pizza is my [MASK] food.', return_tensors='tf') # 토크나이저를 함수처럼 사용

In [4]:
# 결과가 사전 형식으로 구성
# input_ids ; 토큰을 번호로 변환
# tokenizer.cls_token_id = 101 , 문장 전체를 가리키는 특수 토큰
# tokenizer.sep_token_id = 102 , 문장을 나눠주는 특수 토큰
# token_type_ids ; 첫 번째 문장과 두 번째 문장을 구별하는 ids
# attention_mask ; 여러 문장을 동시에 모형에 집어넣었을 때, 문장의 길이를 맞춰주는 패딩을 위한 ids
# 리스트 형태이므로 tensor flow 모형을 사용하므로 return_tensors를 통해 tensor 형식으로 변환
inputs

{'input_ids': <tf.Tensor: shape=(1, 8), dtype=int32, numpy=array([[  101, 10733,  2003,  2026,   103,  2833,  1012,   102]])>, 'token_type_ids': <tf.Tensor: shape=(1, 8), dtype=int32, numpy=array([[0, 0, 0, 0, 0, 0, 0, 0]])>, 'attention_mask': <tf.Tensor: shape=(1, 8), dtype=int32, numpy=array([[1, 1, 1, 1, 1, 1, 1, 1]])>}

In [5]:
# (1, 8, 30522)
# 문장은 1개를 넣었으며, 토큰의 수가 8개
# 토큰 8개에 대해 30522개의 위치에 대해 어떤 토큰이 왔어야 하는가에 대한 예측
# tokenizer.vocab_size = 30522
result = model(inputs)
result

TFMaskedLMOutput(loss=None, logits=<tf.Tensor: shape=(1, 8, 30522), dtype=float32, numpy=
array([[[ -6.5892487,  -6.545541 ,  -6.550171 , ...,  -5.9745717,
          -5.734547 ,  -4.0158253],
        [ -7.9899735,  -7.829558 ,  -7.847027 , ...,  -7.5801945,
          -6.8277617,  -6.485946 ],
        [-12.688318 , -12.100923 , -12.303796 , ..., -10.958023 ,
          -9.478668 , -11.172894 ],
        ...,
        [-10.789202 , -11.01828  , -10.619028 , ...,  -9.310557 ,
          -7.9503617, -12.606783 ],
        [-11.8699875, -11.202813 , -11.640641 , ...,  -8.01551  ,
          -9.46438  ,  -7.81282  ],
        [-16.216097 , -16.040514 , -16.043266 , ..., -16.037786 ,
         -14.760448 , -13.9509945]]], dtype=float32)>, hidden_states=None, attentions=None)

In [6]:
inputs

{'input_ids': <tf.Tensor: shape=(1, 8), dtype=int32, numpy=array([[  101, 10733,  2003,  2026,   103,  2833,  1012,   102]])>, 'token_type_ids': <tf.Tensor: shape=(1, 8), dtype=int32, numpy=array([[0, 0, 0, 0, 0, 0, 0, 0]])>, 'attention_mask': <tf.Tensor: shape=(1, 8), dtype=int32, numpy=array([[1, 1, 1, 1, 1, 1, 1, 1]])>}

In [7]:
logits = result[0]
logits

<tf.Tensor: shape=(1, 8, 30522), dtype=float32, numpy=
array([[[ -6.5892487,  -6.545541 ,  -6.550171 , ...,  -5.9745717,
          -5.734547 ,  -4.0158253],
        [ -7.9899735,  -7.829558 ,  -7.847027 , ...,  -7.5801945,
          -6.8277617,  -6.485946 ],
        [-12.688318 , -12.100923 , -12.303796 , ..., -10.958023 ,
          -9.478668 , -11.172894 ],
        ...,
        [-10.789202 , -11.01828  , -10.619028 , ...,  -9.310557 ,
          -7.9503617, -12.606783 ],
        [-11.8699875, -11.202813 , -11.640641 , ...,  -8.01551  ,
          -9.46438  ,  -7.81282  ],
        [-16.216097 , -16.040514 , -16.043266 , ..., -16.037786 ,
         -14.760448 , -13.9509945]]], dtype=float32)>

In [9]:
import tensorflow as tf
top = tf.math.top_k(logits[0, 4], k=5)
top
# 네 번째 위치한 토큰의 30522개에 대한 확률값 중 가장 높은 것 5개 (5440, 8837, 7216, 6871, 18785)

TopKV2(values=<tf.Tensor: shape=(5,), dtype=float32, numpy=
array([15.272723, 12.503539, 10.711415,  9.428525,  8.84376 ],
      dtype=float32)>, indices=<tf.Tensor: shape=(5,), dtype=int32, numpy=array([ 5440,  8837,  7216,  6871, 18785])>)

In [10]:
tokenizer.decode(top.indices.numpy())
# 위의 번호를 array 형태로 변환한 뒤 토크나이저에 디코드

'favorite favourite comfort preferred staple'

In [11]:
from transformers import pipeline
# 자동으로 [MASK] 찾기 (빈칸 채우기)
pip = pipeline('fill-mask', model='bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'bert.pooler.dense.weight', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [12]:
pip('Pizza is my [MASK] food.')

[{'score': 0.9178980588912964,
  'token': 5440,
  'token_str': 'favorite',
  'sequence': 'pizza is my favorite food.'},
 {'score': 0.05756433680653572,
  'token': 8837,
  'token_str': 'favourite',
  'sequence': 'pizza is my favourite food.'},
 {'score': 0.009590580128133297,
  'token': 7216,
  'token_str': 'comfort',
  'sequence': 'pizza is my comfort food.'},
 {'score': 0.0026588451582938433,
  'token': 6871,
  'token_str': 'preferred',
  'sequence': 'pizza is my preferred food.'},
 {'score': 0.0014816072070971131,
  'token': 18785,
  'token_str': 'staple',
  'sequence': 'pizza is my staple food.'}]

In [13]:
# 반자동으로 [MASK] 찾기 (빈칸 채우기) (원하는 모델과 토크나이저 지정 가능)
from transformers import FillMaskPipeline

In [14]:
pip2 = FillMaskPipeline(model=model, tokenizer=tokenizer)
pip2('Pizza is my [MASK] food.')

[{'score': 0.9178860187530518,
  'token': 5440,
  'token_str': 'favorite',
  'sequence': 'pizza is my favorite food.'},
 {'score': 0.057563528418540955,
  'token': 8837,
  'token_str': 'favourite',
  'sequence': 'pizza is my favourite food.'},
 {'score': 0.009590426459908485,
  'token': 7216,
  'token_str': 'comfort',
  'sequence': 'pizza is my comfort food.'},
 {'score': 0.002658800221979618,
  'token': 6871,
  'token_str': 'preferred',
  'sequence': 'pizza is my preferred food.'},
 {'score': 0.0014815806644037366,
  'token': 18785,
  'token_str': 'staple',
  'sequence': 'pizza is my staple food.'}]

#### KcBERT

In [15]:
# 한국어 데이터에 학습시킨 BERT 모형
from transformers import TFBertForMaskedLM, BertTokenizerFast, FillMaskPipeline

# 한국어 댓글 데이터로 학습
# kcbert-large 는 더 큰 데이터
# pytorch로 학습되어 있으므로 from_pt = True를 통해 tensor flow로 활용
model = TFBertForMaskedLM.from_pretrained('beomi/kcbert-base', from_pt=True)

config.json:   0%|          | 0.00/619 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForMaskedLM: ['cls.predictions.decoder.bias']
- This IS expected if you are initializing TFBertForMaskedLM from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForMaskedLM from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertForMaskedLM were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForMaskedLM for predictions without further training.


In [16]:
tokenizer = BertTokenizerFast.from_pretrained('beomi/kcbert-base')

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/250k [00:00<?, ?B/s]

In [17]:
pipeline = FillMaskPipeline(model=model, tokenizer=tokenizer)
pipeline('피자는 내가 가장 [MASK] 음식이다.')

[{'score': 0.7888962626457214,
  'token': 10015,
  'token_str': '좋아하는',
  'sequence': '피자는 내가 가장 좋아하는 음식이다.'},
 {'score': 0.11531581729650497,
  'token': 13414,
  'token_str': '싫어하는',
  'sequence': '피자는 내가 가장 싫어하는 음식이다.'},
 {'score': 0.031990665942430496,
  'token': 10431,
  'token_str': '먹는',
  'sequence': '피자는 내가 가장 먹는 음식이다.'},
 {'score': 0.015781046822667122,
  'token': 10583,
  'token_str': '원하는',
  'sequence': '피자는 내가 가장 원하는 음식이다.'},
 {'score': 0.011102357879281044,
  'token': 15098,
  'token_str': '사랑하는',
  'sequence': '피자는 내가 가장 사랑하는 음식이다.'}]

#### BERT 미세조정 전처리

In [18]:
import pandas as pd
df = pd.read_csv('./data/imdb.zip')
df.head()

,review,sentiment
0,"A very, very, very slow-moving, aimless movie ...",0
1,Not sure who was more lost - the flat characte...,0
2,Attempting artiness with black & white and cle...,0
3,Very little music or anything to speak of.,0
4,The best scene in the movie was when Gerardo i...,1


In [19]:
# BERT를 이용한 빈칸 채우기
from transformers import TFBertForMaskedLM, BertTokenizerFast, FillMaskPipeline

In [20]:
model = TFBertForMaskedLM.from_pretrained('bert-base-uncased')

All PyTorch model weights were used when initializing TFBertForMaskedLM.

All the weights of TFBertForMaskedLM were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForMaskedLM for predictions without further training.


In [21]:
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

In [22]:
pipeline = FillMaskPipeline(model=model, tokenizer=tokenizer)
pipeline('This is very interesting [MASK].')

[{'score': 0.1292145550251007,
  'token': 5262,
  'token_str': 'indeed',
  'sequence': 'this is very interesting indeed.'},
 {'score': 0.06471988558769226,
  'token': 2592,
  'token_str': 'information',
  'sequence': 'this is very interesting information.'},
 {'score': 0.044466882944107056,
  'token': 2739,
  'token_str': 'news',
  'sequence': 'this is very interesting news.'},
 {'score': 0.03403373062610626,
  'token': 2182,
  'token_str': 'here',
  'sequence': 'this is very interesting here.'},
 {'score': 0.02843305841088295,
  'token': 2205,
  'token_str': 'too',
  'sequence': 'this is very interesting too.'}]

In [23]:
# 전처리
import numpy as np

# 토크나이저의 단어 중, 특수 토큰을 제외하고 일반 토큰만 지정 (100, 102, 0, 101, 103 제외)
# set을 활용해 차집합 계산 
word_tokens = list(set(range(tokenizer.vocab_size)) - set(tokenizer.all_special_ids))

In [24]:
def mask_inputs(input_ids):
    masked = list(input_ids) # 입력에 해당하는 부분
    labels = [-100] * len(masked) # 출력에 해당하는 부분, 숫자에 리스트를 붙혀주면 해당 원소를 반복해줌 (-100을 입력의 길이만큼 반복한 리스트 형성)
                                  # BERT는 -100을 손실 계산에서 제외, 예측을 했을 때 -100이면 손실을 계산하지 않고 손실 계산할 부분만 바꿔줌

    for i in range(1, len(masked) - 1): # input_ids에서 cls token과 sep token을 제외 (무조건 붙기 때문에 맞출 필요가 없음)
        choice1 = np.random.choice(['change', 'not change'], p=[0.15, 0.85]) # change와 not change 중 확률대로 랜덤하게 선택
        if choice1 == 'change': # 토큰 중 하나를 골랐을 때 change 이다
            labels[i] = masked[i] # labels 중 -100이 아닌 토큰의 값이 입력됨
            choice2 = np.random.choice(['mask', 'random', 'same'], p=[0.8, 0.1, 0.1]) # 입력이 들어온대로 예측하는 것이 아닌, 마스크 토큰, 무작위, 같은 단어로 예측
            if choice2 == 'mask':
                masked[i] = tokenizer.mask_token_id # 마스크 토큰으로 배정
            elif choice2 == 'random':
                masked[i] = np.random.choice(word_tokens) # 단어 목록 중 무작위 배정
    return masked, labels
# 논문에 나온 방법대로 처리, 15% 단어만 골라 학습

# masked의 103 이 마스킹된 토큰이며
# labels에 해당하는 토큰의 정답이 有

In [25]:
def mask_inputs(input_ids):
    masked = list(input_ids)
    labels = [-100] * len(masked)
    i = np.random.randint(1, len(masked) - 1) # 문장 전체에서 단어 하나를 무조건 배치 (해당 범위에서 i 하나를 무조건 선택)
    labels[i] = masked[i]
    choice2 = np.random.choice(['mask', 'random', 'same'], p=[0.8, 0.1, 0.1])
    if choice2 == 'mask':
        masked[i] = tokenizer.mask_token_id
    elif choice2 == 'random':
        masked[i] = np.random.choice(word_tokens)
    return masked, labels
# 문장이 짧은 경우 모든 단어가 학습 대상에서 빠질 수 있음(손실을 전혀 계산하지 않음), 문장 중에 한 단어를 골라 학습 대상으로 지정

In [45]:
# TFRecord로 저장
import tensorflow as tf

def int_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=value))

filename = 'bert_train.tfrecord'
with tf.io.TFRecordWriter(filename) as writer:
    for row in df.itertuples(): # 1,000개의 데이터를 하나씩 순환하며 처리
        x = tokenizer(row.review) # 행의 review를 토크나이저에 입력
        masked, labels = mask_inputs(x['input_ids']) # 리뷰의 input_ids를 지정 함수에 입력
        example = tf.train.Example(features=tf.train.Features(feature={
            'input_ids': int_feature(masked), # 마스킹된 ids를 입력
            'token_type_ids': int_feature(x['token_type_ids']), # 원데이터 그대로 입력
            'attention_mask': int_feature(x['attention_mask']), # 원데이터 그대로 입력
            'labels': int_feature(labels) # labels를 입력
        }))
        s = example.SerializeToString() # Serialize 진행
        writer.write(s)

#### BERT 미세조정 Ⅰ

In [27]:
import tensorflow as tf
from transformers import TFBertForMaskedLM, BertTokenizerFast, FillMaskPipeline

model = TFBertForMaskedLM.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

All PyTorch model weights were used when initializing TFBertForMaskedLM.

All the weights of TFBertForMaskedLM were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForMaskedLM for predictions without further training.


In [28]:
# 전처리된 데이터 불러오기
int_seq = tf.io.FixedLenSequenceFeature(shape=(), dtype=tf.int64, allow_missing=True) # 전처리된 데이터의 모든 형태가 동일하므로 변수로 대입
feature_description = { #feature_description 지정
    'input_ids': int_seq,
    'token_type_ids': int_seq,
    'attention_mask': int_seq,
    'labels': int_seq,
}

In [29]:
# 위의 feature_description 을 불러오는 함수 지정
def preproc(example):
    return tf.io.parse_single_example(example, feature_description)

In [30]:
# TFRecord를 .map을 통해 위의 함수를 지정해, TFRecord의 모든 example에 적용
# 문장 32개씩 묶어 하나의 배치로 처리
dataset = tf.data.TFRecordDataset(['bert_train.tfrecord']).map(preproc).padded_batch(32)

In [31]:
# 학습
import tqdm.notebook
import math

opt = tf.keras.optimizers.Adam(learning_rate=3e-5)
for batch in tqdm.notebook.tqdm(dataset, total=math.ceil(1000 / 32)): # 순환을 통해 모델에 배치를 하나씩 넣어줌 # 1000개의 문장을 32개의 batch로 처리 (32개를 처리하면 1 epoch)
    with tf.GradientTape() as tape:
        result = model(batch) # result에 손실값 有, label에서 -100이라고 된 부분은 무시하고 0 이상의 숫자가 들어간 부분만 손실을 계산
                              # 32개의 문장에서 655개의 손실값 계산
        loss = tf.reduce_mean(result['loss']) # 손실값에 대한 평균 계산
    grads = tape.gradient(loss, model.trainable_variables) # 모델의 파라미터를 어떤 방향으로 계산해야 손실이 줄어드는지 계산
    opt.apply_gradients(zip(grads, model.trainable_variables)) # 경사를 적용 (학습)

  0%|          | 0/32 [00:00<?, ?it/s]

In [32]:
# 학습된 모형으로 빈 칸 채우기
pipeline = FillMaskPipeline(model=model, tokenizer=tokenizer)

In [33]:
pipeline('This is very interesting [MASK].')

[{'score': 0.08684153854846954,
  'token': 0,
  'token_str': '[PAD]',
  'sequence': 'this is very interesting.'},
 {'score': 0.005911157000809908,
  'token': 26937,
  'token_str': 'jonny',
  'sequence': 'this is very interesting jonny.'},
 {'score': 0.005225041881203651,
  'token': 25262,
  'token_str': 'environmentally',
  'sequence': 'this is very interesting environmentally.'},
 {'score': 0.003507291665300727,
  'token': 24360,
  'token_str': '##nting',
  'sequence': 'this is very interestingnting.'},
 {'score': 0.0026332736015319824,
  'token': 5456,
  'token_str': 'discovery',
  'sequence': 'this is very interesting discovery.'}]

#### BERT 미세조정 Ⅱ

In [34]:
import tensorflow as tf
from transformers import TFBertForMaskedLM, BertTokenizerFast, FillMaskPipeline

model = TFBertForMaskedLM.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

All PyTorch model weights were used when initializing TFBertForMaskedLM.

All the weights of TFBertForMaskedLM were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForMaskedLM for predictions without further training.


In [35]:
# 모델의 class의 상위 class 확인
# TFBertForMaskedLM은 본인인 동시에 어떤 모델인가?
# 동시에 tensorflow.python.keras.engine.training.Model 이기 때문에 .fit을 통해 학습 가능

model.__class__.mro()

[transformers.models.bert.modeling_tf_bert.TFBertForMaskedLM,
 transformers.models.bert.modeling_tf_bert.TFBertPreTrainedModel,
 transformers.modeling_tf_utils.TFPreTrainedModel,
 keras.src.engine.training.Model,
 keras.src.engine.base_layer.Layer,
 tensorflow.python.module.module.Module,
 tensorflow.python.trackable.autotrackable.AutoTrackable,
 tensorflow.python.trackable.base.Trackable,
 keras.src.utils.version_utils.LayerVersionSelector,
 keras.src.utils.version_utils.ModelVersionSelector,
 transformers.modeling_tf_utils.TFModelUtilsMixin,
 transformers.generation.tf_utils.TFGenerationMixin,
 transformers.utils.hub.PushToHubMixin,
 transformers.modeling_tf_utils.TFMaskedLanguageModelingLoss,
 transformers.modeling_tf_utils.TFCausalLanguageModelingLoss,
 object]

In [46]:
# 전처리된 데이터 불러오기
int_seq = tf.io.FixedLenSequenceFeature(shape=(), dtype=tf.int64, allow_missing=True)
feature_description = {
    'input_ids': int_seq,
    'token_type_ids': int_seq,
    'attention_mask': int_seq,
    'labels': int_seq,
}

In [47]:
def preproc(example):
    x = tf.io.parse_single_example(example, feature_description)
    y = x.pop('labels') # y를 x에서 추출 (x에선 사라짐)
    y = tf.cast(y, tf.int32) # y의 데이터 타입을 정수형으로 변경 (BERT 모델의 출력이 32비트)
    return x, y

In [48]:
dataset = tf.data.TFRecordDataset(['bert_train.tfrecord']).map(preproc).padded_batch(32)

In [49]:
# 학습
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=3e-5),
              loss=model.compute_loss, # 모델이 계산하는 손실함수
              metrics=['accuracy'])

In [ ]:
model.fit(dataset) #.fit 함수를 통해 편하게 학습 가능

In [ ]:
# 학습된 모형으로 빈 칸 채우기
pipeline = FillMaskPipeline(model=model, tokenizer=tokenizer)
pipeline('This is very interesting [MASK].')

#### 감성 분석을 위한 전처리

In [51]:
import pandas as pd
nsmc = pd.read_csv('./data/ratings_train.txt', sep='\t')
nsmc.head()
# 네이버 영화평 감성분석 데이터 (tap으로 구분된 파일)

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [52]:
# 토크나이저
import tensorflow as tf
from transformers import BertTokenizerFast

tokenizer = BertTokenizerFast.from_pretrained('beomi/kcbert-base')

In [53]:
# 분할
small = nsmc.iloc[:10000]
# train data와 test 데이터 분할을 위해 분할
# 15만건의 큰 데이터이기 때문에도 분할

In [54]:
from sklearn.model_selection import train_test_split

idx_train, idx_test = train_test_split(small.index, test_size=0.2, random_state=42)
# 2000개 test, 8000개 train 분할

In [55]:
# TFRecord로 저장
def int_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=value))

In [56]:
filename = 'test_nsmc.tfrecord'
df = small.loc[idx_test]

In [57]:
# test tfrecord 저장
with tf.io.TFRecordWriter(filename) as writer:
    for row in df.itertuples():
        x = tokenizer(row.document)
        example = tf.train.Example(features=tf.train.Features(feature={
            'input_ids': int_feature(x['input_ids']),
            'token_type_ids': int_feature(x['token_type_ids']),
            'attention_mask': int_feature(x['attention_mask']),
            'labels': int_feature([row.label])
        }))
        s = example.SerializeToString()
        writer.write(s)

In [58]:
# train tfrecord 저장
filename = 'train_nsmc.tfrecord'
df = small.loc[idx_train]

with tf.io.TFRecordWriter(filename) as writer:
    for row in df.itertuples():
        x = tokenizer(row.document) # 행의 리뷰 내용을 토크나이저에 입력
        example = tf.train.Example(features=tf.train.Features(feature={ # x를 출력해보면 input_ids, oken_type_ids, attention_mask 가 나옴
            'input_ids': int_feature(x['input_ids']), # 원래 리스트 형태이기 때문에 그대로 입력 (이하 동일)
            'token_type_ids': int_feature(x['token_type_ids']),
            'attention_mask': int_feature(x['attention_mask']),
            'labels': int_feature([row.label]) # 원데이터의 label 사용, row.label은 정수 하나이기 때문에 반드시 리스트 형태로 감싸서 넣어줘야 함
        }))
        s = example.SerializeToString()
        writer.write(s)

#### 감성분석 미세조정 Ⅰ (경사하강법)

In [59]:
# 연속으로 구성된 긍부정 분류 (=문장) : SequenceClassification
import tensorflow as tf
from transformers import TFBertForSequenceClassification, BertTokenizerFast

model = TFBertForSequenceClassification.from_pretrained('beomi/kcbert-base', from_pt=True)
tokenizer = BertTokenizerFast.from_pretrained('beomi/kcbert-base')

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [60]:
# 데이터 불러오기
int_seq = tf.io.FixedLenSequenceFeature(shape=(), dtype=tf.int64, allow_missing=True)
int_value = tf.io.FixedLenFeature(shape=(), dtype=tf.int64)
feature_description = {
    'input_ids': int_seq,
    'token_type_ids': int_seq,
    'attention_mask': int_seq,
    'labels': int_value # 위의 셋은 토큰화되서 문장을 쪼개 놓기 때문에 길이가 있지만, label은 0 아니면 1이기 때문에 길이가 없어서 LenFeature를 통해 정의
}

In [61]:
def preproc(example):
    return tf.io.parse_single_example(example, feature_description)

In [62]:
train_dataset = tf.data.TFRecordDataset(['train_nsmc.tfrecord']).map(preproc).padded_batch(32)
test_dataset = tf.data.TFRecordDataset(['test_nsmc.tfrecord']).map(preproc).padded_batch(32)

In [63]:
next(iter(test_dataset)) # Serialize 된 상태라 parsing을 해줘야 함, 위의 식에서 진행

{'attention_mask': <tf.Tensor: shape=(32, 74), dtype=int64, numpy=
 array([[1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        ...,
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0]], dtype=int64)>,
 'input_ids': <tf.Tensor: shape=(32, 74), dtype=int64, numpy=
 array([[    2,  9376, 12746, ...,     0,     0,     0],
        [    2,  7992,  2500, ...,     0,     0,     0],
        [    2, 15801,    95, ...,     0,     0,     0],
        ...,
        [    2, 28684,  1594, ...,     0,     0,     0],
        [    2,  2878, 29809, ...,     0,     0,     0],
        [    2,  2346,  4227, ...,     0,     0,     0]], dtype=int64)>,
 'labels': <tf.Tensor: shape=(32,), dtype=int64, numpy=
 array([0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 0, 1], dtype=int64)>,
 'token_type_ids': <tf.Tensor: shape=(32, 74), dtype=int64, numpy=
 array([[0, 0, 0, ...,

In [64]:
batch = next(iter(test_dataset))
batch # 문장 32개가 묶임

{'attention_mask': <tf.Tensor: shape=(32, 74), dtype=int64, numpy=
 array([[1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        ...,
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0]], dtype=int64)>,
 'input_ids': <tf.Tensor: shape=(32, 74), dtype=int64, numpy=
 array([[    2,  9376, 12746, ...,     0,     0,     0],
        [    2,  7992,  2500, ...,     0,     0,     0],
        [    2, 15801,    95, ...,     0,     0,     0],
        ...,
        [    2, 28684,  1594, ...,     0,     0,     0],
        [    2,  2878, 29809, ...,     0,     0,     0],
        [    2,  2346,  4227, ...,     0,     0,     0]], dtype=int64)>,
 'labels': <tf.Tensor: shape=(32,), dtype=int64, numpy=
 array([0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 0, 1], dtype=int64)>,
 'token_type_ids': <tf.Tensor: shape=(32, 74), dtype=int64, numpy=
 array([[0, 0, 0, ...,

In [65]:
# 학습 전 성능 테스트
import tqdm.notebook
import math

a = 0
for batch in tqdm.notebook.tqdm(test_dataset, total=math.ceil(2000 / 32)):
    result = model(batch)
    equals = tf.math.argmax(result['logits'], axis=1) == batch['labels'] # logits이 큰 쪽으로 예측 (0 혹은 1) # 모델을 통해 나온 예측값과 batch의 label(정답)과 비교
    equals = equals.numpy() # equals.sum()/len(equals) = 배치에 들어있는 문장 중 맞춘 개수
    a += equals.sum()
    
a / 2000

  0%|          | 0/63 [00:00<?, ?it/s]

0.469

In [66]:
# 학습
opt = tf.keras.optimizers.Adam(learning_rate=3e-5)

for batch in tqdm.notebook.tqdm(train_dataset, total=math.ceil(8000/32)):
    with tf.GradientTape() as tape:
        result = model(batch)
        loss = tf.reduce_mean(result['loss'])
    grads = tape.gradient(loss, model.trainable_variables)
    opt.apply_gradients(zip(grads, model.trainable_variables))

  0%|          | 0/250 [00:00<?, ?it/s]

In [67]:
# 학습 후 성능 테스트
a = 0
for batch in tqdm.notebook.tqdm(test_dataset, total=math.ceil(2000 / 32)):
    result = model(batch)
    equals = tf.math.argmax(result['logits'], axis=1) == batch['labels']
    equals = equals.numpy()
    a += equals.sum()
a / 2000

  0%|          | 0/63 [00:00<?, ?it/s]

0.8585

#### 감성분석 미세조정 Ⅱ

In [68]:
import tensorflow as tf
from transformers import TFBertForSequenceClassification, BertTokenizerFast

model = TFBertForSequenceClassification.from_pretrained('beomi/kcbert-base', from_pt=True)
tokenizer = BertTokenizerFast.from_pretrained('beomi/kcbert-base')

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [69]:
model.__class__.mro()
# 상위 class 중  tensorflow.python.keras.engine.training.Model 有
# fit 함수 사용 가능

[transformers.models.bert.modeling_tf_bert.TFBertForSequenceClassification,
 transformers.models.bert.modeling_tf_bert.TFBertPreTrainedModel,
 transformers.modeling_tf_utils.TFPreTrainedModel,
 keras.src.engine.training.Model,
 keras.src.engine.base_layer.Layer,
 tensorflow.python.module.module.Module,
 tensorflow.python.trackable.autotrackable.AutoTrackable,
 tensorflow.python.trackable.base.Trackable,
 keras.src.utils.version_utils.LayerVersionSelector,
 keras.src.utils.version_utils.ModelVersionSelector,
 transformers.modeling_tf_utils.TFModelUtilsMixin,
 transformers.generation.tf_utils.TFGenerationMixin,
 transformers.utils.hub.PushToHubMixin,
 transformers.modeling_tf_utils.TFSequenceClassificationLoss,
 object]

In [70]:
# 데이터 불러오기
int_seq = tf.io.FixedLenSequenceFeature(shape=(), dtype=tf.int64, allow_missing=True)
int_value = tf.io.FixedLenFeature(shape=(), dtype=tf.int64)
feature_description = {
    'input_ids': int_seq,
    'token_type_ids': int_seq,
    'attention_mask': int_seq,
    'labels': int_value
}

In [71]:
def preproc(example): # 전처리 함수
    x = tf.io.parse_single_example(example, feature_description)
    y = x.pop('labels')
    return x, tf.cast(y, tf.int32) # y를 32비트로 바꿈

In [72]:
train_dataset = tf.data.TFRecordDataset(['train_nsmc.tfrecord']).map(preproc).padded_batch(32)
test_dataset = tf.data.TFRecordDataset(['test_nsmc.tfrecord']).map(preproc).padded_batch(32)

In [73]:
# 모형 설정
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=3e-5), loss=model.compute_loss, metrics=['accuracy'])
# model compile, 손실함수는 모델이 자체적으로 갖고 있는 손실함수 사용

In [ ]:
# 학습 전 성능 테스트
model.evaluate(test_dataset)

In [ ]:
# 학습
model.fit(train_dataset)
# 일일이 기울기 구하고, optimizer 적용 필요 X

In [ ]:
# 학습 후 성능 테스트
model.evaluate(test_dataset)

#### 질의 응답 실습 (QA Task를 BERT를 통해 실습)

In [75]:
from transformers import pipeline
qa = pipeline('question-answering') # question-answering 모델을 다운로드

No model was supplied, defaulted to distilbert-base-cased-distilled-squad and revision 626af31 (https://huggingface.co/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [76]:
context = """
Youn Yuh-jung (born June 19, 1947) is a South Korean actress, whose career in film and television spans over five decades. 
Her accolades include an Academy Award.
"""
# 문장 입력, """ """ 을 통해 여러 줄로 된 텍스트를 입력할 수 있음

In [77]:
# question에 질문을 입력하고, context를 통해 텍스트를 입력
# 위 텍스트에서 직업에 해당하는 위치를 보여줌
# 54-61 : 글자의 위치
# score = 확률을 통해 계산한 정답일 가능성
# job 이라는 단어가 없다 = 문구가 똑같지 않더라도 텍스트에서 필요한 부분을 추출해서 답해줌
qa(question="What is her job?", context=context)

{'score': 0.7455781698226929, 'start': 54, 'end': 61, 'answer': 'actress'}

In [78]:
qa(question="What did she win?", context=context)

{'score': 0.6411082744598389,
 'start': 150,
 'end': 163,
 'answer': 'Academy Award'}

In [79]:
qa(question="When was she born?", context=context)

{'score': 0.9665399789810181,
 'start': 21,
 'end': 34,
 'answer': 'June 19, 1947'}

In [80]:
# 모형을 지정해서 pipeline 만들기

from transformers import AutoTokenizer, TFAutoModelForQuestionAnswering, QuestionAnsweringPipeline
# AutoTokenizer나 AutoModel은 사전학습모델을 호출할 때, 모델이 어떤 class를 사용하고 있는지 정보가 있기 때문에, BERT, GPT2 등을 지정하지 않아도 됨

In [81]:
# 미세조정 되지 않은 모형

model = TFAutoModelForQuestionAnswering.from_pretrained("bert-base-uncased")
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
# Auto 지만 BERT인 것을 알고 있기 때문에 BERTTokenizer를 호출했어도 된다

All PyTorch model weights were used when initializing TFBertForQuestionAnswering.

Some weights or buffers of the TF 2.0 model TFBertForQuestionAnswering were not initialized from the PyTorch model and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [82]:
# Pipeline에 모델과 토크나이저를 입력
# BERT 기본 모형이기 때문에 QA를 할 수 있도록 training 되어 있지 않음
qa = QuestionAnsweringPipeline(model=model, tokenizer=tokenizer)

In [83]:
qa(question="What is her job?", context=context)

{'score': 0.002011405536904931, 'start': 54, 'end': 62, 'answer': 'actress,'}

In [84]:
qa(question="What did she win?", context=context)

{'score': 0.0020648217760026455, 'start': 54, 'end': 62, 'answer': 'actress,'}

In [85]:
# 트레이닝이 되어 있지 않은 모델을 사용하면 전혀 답을 하지 못함
qa(question="When was she born?", context=context)

{'score': 0.0019660096149891615, 'start': 54, 'end': 62, 'answer': 'actress,'}

In [86]:
# 미세조정 된 모형

model = TFAutoModelForQuestionAnswering.from_pretrained("distilbert-base-cased-distilled-squad")
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-cased-distilled-squad')
# BERT를 압축시켜 놓고(distiled), 작은 (base), 대소문자 구분한 (cased), 스탠포드 대학에서 만든 QA dataset (squad)
# BERT 모형을 squad data에 미세조정

All PyTorch model weights were used when initializing TFDistilBertForQuestionAnswering.

All the weights of TFDistilBertForQuestionAnswering were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForQuestionAnswering for predictions without further training.


In [87]:
qa = QuestionAnsweringPipeline(model=model, tokenizer=tokenizer)

In [88]:
qa(question="What is her job?", context=context)

{'score': 0.7455788850784302, 'start': 54, 'end': 61, 'answer': 'actress'}

In [89]:
qa(question="What did she win?", context=context)

{'score': 0.6411080956459045,
 'start': 150,
 'end': 163,
 'answer': 'Academy Award'}

In [90]:
qa(question="When was she born?", context=context)

{'score': 0.966539740562439, 'start': 21, 'end': 34, 'answer': 'June 19, 1947'}

In [91]:
# 수동 (미세조정을 위한)
import tensorflow as tf

# 질문 생성
# 토크나이저에 질문을 입력 (question과 context의 순서는 사실 무관 = BERT 모형의 특성)
# 모델 자체가 Q-A 순으로 학습되어 있기 때문에 토크나이징할 때, Q-A 순으로 입력
# add_special_tokens
question = 'What did she win?'
inputs = tokenizer(question, context, add_special_tokens=True, return_tensors="tf")

# 101 = CLS 토큰
# 102 = SEP 토큰
# 48개의 토큰
inputs

{'input_ids': <tf.Tensor: shape=(1, 48), dtype=int32, numpy=
array([[  101,  1327,  1225,  1131,  1782,   136,   102,  1192,  1179,
        10684,  1324,   118,   179,  4380,   113,  1255,  1340,  1627,
          117,  3138,   114,  1110,   170,  1375,  3947,  3647,   117,
         2133,  1578,  1107,  1273,  1105,  1778, 15533,  1166,  1421,
         4397,   119,  1430,   170, 14566, 19872,  1511,  1126,  2127,
         1698,   119,   102]])>, 'attention_mask': <tf.Tensor: shape=(1, 48), dtype=int32, numpy=
array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1]])>}

In [92]:
tokenizer.decode(inputs['input_ids'].numpy()[0])

'[CLS] What did she win? [SEP] Youn Yuh - jung ( born June 19, 1947 ) is a South Korean actress, whose career in film and television spans over five decades. Her accolades include an Academy Award. [SEP]'

In [93]:
# input 그대로 모델에 넣으면 됨
outputs = model(inputs)

In [94]:
# 입력한 48개의 토큰 중, softmax 함수에 들어가면 확률로 변환되는데, 질문의 답변이 어느 곳에 시작할 것인가?
# 값이 높을수록 답변이 시작하는 곳일 확률이 높음
outputs.start_logits

<tf.Tensor: shape=(1, 48), dtype=float32, numpy=
array([[-3.2587407 , -4.115965  , -6.1125135 , -5.7539034 , -6.0889006 ,
        -3.914919  , -4.3926554 , -2.0310116 , -5.905585  , -3.8264377 ,
        -6.541384  , -6.294675  , -5.6275797 , -5.9331894 , -5.194643  ,
        -3.7380962 , -2.697707  , -3.7542107 , -5.712838  , -2.1804962 ,
        -4.863432  , -4.760373  , -3.570895  , -0.47888386, -2.6021426 ,
        -1.3134253 , -6.2036495 , -5.035602  , -4.1924896 , -6.55541   ,
        -2.4696975 , -7.3524957 , -3.1218536 , -5.6631937 , -4.4715004 ,
        -3.9809253 , -5.159875  , -2.9622166 ,  2.0888474 , -0.665982  ,
        -5.286133  , -4.834054  , -2.0916827 ,  8.230597  ,  8.933442  ,
         1.8657247 , -2.8914907 , -4.3925533 ]], dtype=float32)>

In [95]:
# 어디서 답변이 끝날 것인가?
outputs.end_logits

<tf.Tensor: shape=(1, 48), dtype=float32, numpy=
array([[-0.7095605 , -3.614283  , -6.2600956 , -5.9759374 , -6.0352654 ,
        -4.777353  , -4.5723333 , -4.9856696 , -4.3647704 , -4.8074465 ,
        -4.402199  , -5.2577233 , -5.4835253 , -1.9583439 , -4.912096  ,
        -4.9395914 , -4.1950364 , -3.3444967 , -3.7767985 , -1.870179  ,
        -1.4572221 , -5.2039084 , -5.880264  , -4.244818  ,  0.21101947,
        -0.7073603 , -1.2311841 , -5.6432405 , -4.7058434 , -7.4447846 ,
        -4.0362253 , -7.16896   , -2.56027   , -6.0628514 , -6.265037  ,
        -4.800591  , -1.379861  ,  0.35461897, -3.190801  , -6.381058  ,
        -5.4437137 , -1.0014491 , -3.896157  , -0.3924904 ,  3.264592  ,
         9.571489  ,  6.337932  , -4.5721397 ]], dtype=float32)>

In [96]:
# 시작 위치의 확률 중 가장 높은 곳을 확인
# axis = 0, 행을 바꿔가며 가장 큰 값이 있는 행을 찾음
# 열을 찾아야함 -> 이후 결과값을 numpy 형식으로 바꾼 뒤 [0]을 통해 위치에 해당하는 숫자만 출력
start = tf.argmax(outputs.start_logits, axis=1).numpy()[0]

In [97]:
# 위와 동일, 범위 지정에 있어 끝나는 토큰을 포함시키기 위해 +1을 해줌
end = tf.argmax(outputs.end_logits, axis=1).numpy()[0] + 1

In [98]:
# 0번 행에서 시작 위치부터 끝나는 위치까지 뽑아서 디코드를 진행해 답변을 문장으로 바꿔줌
tokenizer.decode(inputs['input_ids'][0, start:end])

'Academy Award'

#### 한국어 질의응답

In [133]:
# 모델 호출 (Pytorch 버젼의 모델)
from transformers import AutoTokenizer, TFAutoModelForQuestionAnswering, QuestionAnsweringPipeline
import tensorflow as tf
model = TFAutoModelForQuestionAnswering.from_pretrained('beomi/kcbert-base', from_pt=True)

All PyTorch model weights were used when initializing TFBertForQuestionAnswering.

Some weights or buffers of the TF 2.0 model TFBertForQuestionAnswering were not initialized from the PyTorch model and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [134]:
# 토크나이저 호출
tokenizer = AutoTokenizer.from_pretrained('beomi/kcbert-base')

In [101]:
qa = QuestionAnsweringPipeline(model=model, tokenizer=tokenizer)

In [102]:
context = '윤여정은 1966년 연극 배우로 연기 경력을 시작하였고, 2021년 영화 《미나리》의 순자 역으로 아카데미 여우조연상을 수상했다.'
question = '윤여정의 직업은?'

In [103]:
inputs= tokenizer(context, question, return_tensors='tf') # 원활한 전처리를 위해 context-question 순으로 구성

In [104]:
result = model(inputs)

In [105]:
import tensorflow as tf

start = tf.argmax(result.start_logits, axis=1).numpy()[0]
end = tf.argmax(result.end_logits, axis=1).numpy()[0] + 1

tokenizer.decode(inputs['input_ids'][0, start:end])

'[CLS] 윤여정은 1966년 연극 배우로 연기 경력을 시작하였고, 2021년 영화 [UNK] 미'

In [106]:
# 현재 모형은 QA training이 안 되어 있다
# 위와 동일하게 엉망
qa(question='윤여정의 직업은?', context=context)

{'score': 0.004640113562345505,
 'start': 21,
 'end': 45,
 'answer': '경력을 시작하였고, 2021년 영화 《미나리'}

#### KorQUAD 전처리

In [107]:
# 한국어 QA Dataset인 KorQUAD를 학습
import urllib.request
urllib.request.urlretrieve('https://korquad.github.io/dataset/KorQuAD_v1.0_train.json', 'korquad.json')
# KorQUAD 사이트에서 링크 주소 복사를 한 뒤, urlib.request 모듈을 통해 해당 복사 주소를 다운받을 수 있으며, 다운받을 파일 이름 지정

('korquad.json', <http.client.HTTPMessage at 0x1f252bcf160>)

In [108]:
# json 형식의 파일을 읽기 위해 import
import json

# jsom 파일을 읽기
korquad = json.load(open('korquad.json', encoding='utf8'))

In [109]:
# dictionary 형태로 되어 있는데, version과 data로 구성
# data는 리스트 형태로 되어있기 때문에 [0] 을 통해 첫 번째 예제가 나옴
# paragraphs 를 통해 문단의 내용 (context) 과 질문 (qas) 과 답 (answers)으로 구성되어 있음
korquad['data'][0].keys()

dict_keys(['paragraphs', 'title'])

In [110]:
korquad['data'][0]['title']

'파우스트_서곡'

In [111]:
item = korquad['data'][0]
para = item['paragraphs'][0]
para['context']

'1839년 바그너는 괴테의 파우스트을 처음 읽고 그 내용에 마음이 끌려 이를 소재로 해서 하나의 교향곡을 쓰려는 뜻을 갖는다. 이 시기 바그너는 1838년에 빛 독촉으로 산전수전을 다 걲은 상황이라 좌절과 실망에 가득했으며 메피스토펠레스를 만나는 파우스트의 심경에 공감했다고 한다. 또한 파리에서 아브네크의 지휘로 파리 음악원 관현악단이 연주하는 베토벤의 교향곡 9번을 듣고 깊은 감명을 받았는데, 이것이 이듬해 1월에 파우스트의 서곡으로 쓰여진 이 작품에 조금이라도 영향을 끼쳤으리라는 것은 의심할 여지가 없다. 여기의 라단조 조성의 경우에도 그의 전기에 적혀 있는 것처럼 단순한 정신적 피로나 실의가 반영된 것이 아니라 베토벤의 합창교향곡 조성의 영향을 받은 것을 볼 수 있다. 그렇게 교향곡 작곡을 1839년부터 40년에 걸쳐 파리에서 착수했으나 1악장을 쓴 뒤에 중단했다. 또한 작품의 완성과 동시에 그는 이 서곡(1악장)을 파리 음악원의 연주회에서 연주할 파트보까지 준비하였으나, 실제로는 이루어지지는 않았다. 결국 초연은 4년 반이 지난 후에 드레스덴에서 연주되었고 재연도 이루어졌지만, 이후에 그대로 방치되고 말았다. 그 사이에 그는 리엔치와 방황하는 네덜란드인을 완성하고 탄호이저에도 착수하는 등 분주한 시간을 보냈는데, 그런 바쁜 생활이 이 곡을 잊게 한 것이 아닌가 하는 의견도 있다.'

In [112]:
# 답변의 시작 위치가 나옴
# 전처리 시, 질문을 앞에 놓을 시, 시작 위치에 질문의 길이를 더해줘야 하기 때문에, 간단하기 진행하기 위해 질문을 뒤에 넣고 전처리
qas = para['qas'][0]
qas

{'answers': [{'text': '교향곡', 'answer_start': 54}],
 'id': '6566495-0-0',
 'question': '바그너는 괴테의 파우스트를 읽고 무엇을 쓰고자 했는가?'}

In [113]:
qas['question']

'바그너는 괴테의 파우스트를 읽고 무엇을 쓰고자 했는가?'

In [114]:
qas['answers'][0]['text']

'교향곡'

In [115]:
tokenizer.model_max_length # KcBERT가 300개의 토큰까지만 처리할 수 있음

300

In [116]:
inputs = tokenizer(
    para['context'], 
    qas['question'], 
    truncation=True, # 길이가 길면 자름
    max_length=tokenizer.model_max_length, # 길이를 지정해주지 않아도 내부적으로 설정되어 있지만, max_length를 통해 최대 길이를 넘어가면 자름 
    return_tensors='tf')
# 올바르게 처리하려면, context가 너무 길면, context의 길이를 자르던가
# question을 앞으로 놓고, answer의 start 부분을 처리해야 함

In [117]:
# 학습을 위해서 시작 토큰과 끝나는 토큰 위치를 추가해야 함
inputs

{'input_ids': <tf.Tensor: shape=(1, 300), dtype=int32, numpy=
array([[    2,  8601,  4633, 29697,  1480,  4313,  4538,  4008,   336,
         4065,  4042,  3231, 23243,  4104,  4027,  8793, 13985,   391,
         9132,  4113, 10966, 11728, 12023, 14657,  4091,  8598, 16639,
          341,  4573,  4771,  4027,  2139,  8478, 14416,   214,  8202,
           17,  2451, 13007,  1480,  4313,  4538,  4008,  8601,  4633,
        22903,  4113,  1676,   868,  4913,  7965,  1789,  4203,  4110,
        15031,   786,   250,  4057, 10878,  4007,  2593,  4094,  4128,
        10289,  4113, 10958,  4062,  9511,  1355,  4600,  4103,  4775,
         5602, 10770,  4180, 26732,  3231, 23243,  4104,  4042,  2015,
         4012,  4113,  9198,  8763,  8129,    17, 10384, 23008,  7971,
         2170,  4408,  4011,  4147,  4042, 17015,  4091, 23008, 21056,
         4165,   323,  4175,  4158, 11413,  2273,  4043,  7966,  1543,
         4775,  4170,  4042,   341,  4573,  4771,    28,  4566,  4027,
        10599, 

In [118]:
q = qas['answers'][0]
start_char = q['answer_start'] # 답변의 시작 부분
end_char = start_char + len(q['text']) - 1 # 답변의 끝 부분 (모델에 넣기 위해서는 -1 을 해야함)
start = inputs.char_to_token(0, start_char) 
# 현재는 글자의 위치이므로 토큰의 위치로 변환해줘야함 (준단어 토큰화를 했기 때문에)
# 위에서 나오는 번호의 글자에 해당하는 토큰이 몇 번째 토큰인가 계산
end = inputs.char_to_token(0, end_char)
# 위와 동일한 방식

In [119]:
tokenizer.decode(inputs['input_ids'].numpy()[0, start:end+1])

'교향곡'

In [120]:
inputs['start_positions'] = [start] # 시작 위치 지정(리스트 형태 = tfrecord 형태로 저장할 때, 저장이 편하기 위해)

In [121]:
# 하나의 예시를 처리
# 모든 예시에 대해 처리한 것을 tfrecord 형태로 만들어주기 위해 반복을 위한 함수를 만듦
inputs['end_positions'] = [end] # 끝 위치 지정(리스트 형태)

In [122]:
def int_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=value))

In [123]:
def make_inputs(context, qas):
    inputs = tokenizer(
        context, 
        qas['question'], 
        truncation=True, 
        max_length=tokenizer.model_max_length)
    q = qas['answers'][0]
    start_char = q['answer_start']
    end_char = start_char + len(q['text']) - 1
    start = inputs.char_to_token(0, start_char)
    end = inputs.char_to_token(0, end_char)
    inputs['start_positions'] = [start]
    inputs['end_positions'] = [end]
    return inputs

In [124]:
n = 0
filename = 'korquad.tfrecord'
with tf.io.TFRecordWriter(filename) as writer:
    for item in korquad['data']:
        for para in item['paragraphs']:
            context = para['context']
            for qas in para['qas']:
                inputs = make_inputs(context, qas)
                if inputs['start_positions'][0] and inputs['end_positions'][0]: 
                # 300 토큰이 넘으면 truncate 되기 때문에 start와 end position이 input 안에 포함되어 있을 때만 저장
                    feature = {k: int_feature(v) for k, v in inputs.items()} # inputs가 dictionary이기 때문에, dictionary의 모든 값에 대해 적용
                    example = tf.train.Example(features=tf.train.Features(feature=feature))
                    s = example.SerializeToString() # Example을 문자로 변환
                    writer.write(s)
                    n += 1

In [125]:
# 사례의 수
n

57653

#### 한국어 QA 미세조정

In [126]:
# 불러오기
int_seq = tf.io.FixedLenSequenceFeature(shape=(), dtype=tf.int64, allow_missing=True)
int_value = tf.io.FixedLenFeature(shape=(), dtype=tf.int64)
feature_description = {
    'input_ids': int_seq,
    'token_type_ids': int_seq,
    'attention_mask': int_seq,
    'start_positions': int_value,
    'end_positions': int_value
}
# 토큰은 sequence 로, start와 end position 은 value 로 되어있음

In [127]:
# 전처리 함수 만들기
def preproc(example):
    example = tf.io.parse_single_example(example, feature_description)
    example = {k : tf.cast(v, tf.int32) for k, v in example.items()} # 64비트 정수로 저장되어 있는 것을 32비트로 변경
    return example

In [128]:
# OOM 발생 시, 문장 길이를 줄이던지, batch size를 줄이던지 해야함
dataset = tf.data.TFRecordDataset(['korquad.tfrecord']).map(preproc).padded_batch(8)

In [129]:
batch = next(iter(dataset))

In [130]:
result = model(batch)

In [ ]:
# 미세조정
import tqdm.notebook
import math

n = 57653
opt = tf.keras.optimizers.Adam(learning_rate=5e-5) # 학습률을 매우 낮게 지정

for batch in tqdm.notebook.tqdm(dataset, total=math.ceil(n / 8)): # dataset에서 batch를 하나씩 가져옴
    with tf.GradientTape() as tape: 
        result = model(batch) # 모델에 8개 example을 동시에 넣어서 계산
        loss = tf.reduce_mean(result['loss']) # loss의 평균을 도출
    grads = tape.gradient(loss, model.trainable_variables) # 손실값과 파라미터의 경사를 구해 손실이 줄어드는 방향을 구함
    opt.apply_gradients(zip(grads, model.trainable_variables)) # 손실을 줄어드는 방향으로 파라미터 업데이트

model.save_pretrained('./model/bert/kcbert-base-korquad')

In [138]:
# 학습된 모델 호출 (저장된)
model = TFAutoModelForQuestionAnswering.from_pretrained('./model/bert/kcbert-base-korquad')

All model checkpoint layers were used when initializing TFBertForQuestionAnswering.

All the layers of TFBertForQuestionAnswering were initialized from the model checkpoint at ./model/bert/kcbert-base-korquad.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForQuestionAnswering for predictions without further training.


In [139]:
# 사용해보기
tokenizer = AutoTokenizer.from_pretrained('beomi/kcbert-base')

In [140]:
context = '윤여정은 1966년 연극 배우로 연기 경력을 시작하였고, 2021년 영화 《미나리》의 순자 역으로 아카데미 여우조연상을 수상했다.'

In [141]:
def qa(question, context):
    inputs = tokenizer(context, question, add_special_tokens=True, return_tensors="tf")
    outputs = model(inputs)
    start = tf.argmax(outputs.start_logits, axis=1).numpy()[0]
    end = tf.argmax(outputs.end_logits, axis=1).numpy()[0]
    return tokenizer.decode(inputs['input_ids'][0, start:end+1])

In [142]:
qa('윤여정은 언제 데뷔했나?', context)

'2021년'

In [143]:
qa('윤여정의 직업은?', context)

'연극 배우'

In [144]:
qa('윤여정이 받은 상은?', context)

'아카데미 여우조연상을'

In [145]:
qa('윤여정의 출역작은?', context)

'미나리'